In [2]:
%pip install datasets
%pip install nltk
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Part 0. Data Prepraration

In [3]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset ['train']
validation_dataset = dataset ['validation']
test_dataset = dataset ['test']

c:\Users\JJWX\anaconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Part 1. Preparing Word Embeddings

#### (a) What is the size of the vocabulary formed from your training data?

In [4]:
import nltk
import numpy as np
nltk.download('punkt')
vocab = set()
for text in train_dataset['text']:
    ls = nltk.word_tokenize(text)
    for word in ls:
        if word.isalpha(): vocab.add(word)
#Size of the vocabulary: 15812

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JJWX\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
print("Size of the vocabulary:", len(vocab))

Size of the vocabulary: 15812


#### (b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

In [6]:
import gensim.downloader as api
for key in api.info()['models'].keys():
    print(key)

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [7]:
model = api.load("glove-wiki-gigaword-100")

In [8]:
oov_words = set()
for word in vocab:
    if word not in model:
        oov_words.add(word)

print("Number of OOV words:", len(oov_words))

Number of OOV words: 550


#### (c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.

In [9]:
# Group any words that are not in the model into a single token
def wordtovec(word):
    if word in model:
        return model[word]
    else:
        return np.zeros(model.vector_size)

# Part 2. Model Training & Evaluation - RNN

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import random

In [11]:
vocab_size = len(model.index_to_key) + 1
embedding_dim = model.vector_size
word_index = {word: index+1 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [12]:
for word, idx in word_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]

In [13]:
def tokenize(text, word_index):
    ls = nltk.word_tokenize(text)
    return [word_index[word] for word in ls if word in word_index]

X_train = [tokenize(text, word_index) for text in train_dataset['text']]
X_val = [tokenize(text, word_index) for text in validation_dataset['text']]
X_test = [tokenize(text, word_index) for text in test_dataset['text']]
max_length = max(len(seq) for seq in X_train)

In [14]:
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

In [15]:
y_train = np.array(train_dataset['label'])
y_val = np.array(validation_dataset['label'])
y_test = np.array(test_dataset['label'])

Model Training - Grid Search

In [32]:
from tensorflow.keras.callbacks import Callback
best_accuracy = {}
class CustomCallback(Callback):
    accuracy = 0
    cur_key = ""
    epochs = 0
    optimizer = ""
    batch_size = 0
    best_model = None
    lr = 0
    def on_train_begin(self, logs=None):
        self.accuracy = 0

    def on_train_end(self, logs=None):
        global best_accuracy
        if self.accuracy > best_accuracy.get("accuracy", 0):
            best_accuracy = {
                "accuracy": self.accuracy,
                "epoch": self.epochs,
                "optimizer": self.optimizer,
                "batch_size": self.batch_size,
                "lr": self.lr
            }
            print("Saved best accuracy for current run:", self.accuracy, "at epoch", self.epochs)
            self.best_model.save(filepath="best_model.keras")
        print("Run completed on:")
        print(self.cur_key)
        print("Best accuracy for current run:", self.accuracy, "at epoch", self.epochs)
        print("Training ended")
    
    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs['val_accuracy']
        if val_accuracy > self.accuracy:
            self.accuracy = val_accuracy
            self.epochs = epoch
            self.best_model = self.model

    def set_key(self, optimizer, batch_size, lr):
        self.optimizer = optimizer
        self.batch_size = batch_size
        self.lr = lr
        self.cur_key = f"optimizer: {optimizer}, batch_size: {batch_size}, lr: {lr}"

In [33]:
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    custom_callback = CustomCallback()
    custom_callback.set_key(optimizer, batch_size, lr)
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[custom_callback, early_stopping]
    )
    return model, history

In [34]:
for batch_size in [16, 32, 64, 128]:
    for lr in [0.005, 0.01, 0.05, 0.1]:
        for optimizer in ['adam', 'sgd', 'rmsprop', 'adagrad']:
            train_model(optimizer, 100, batch_size, lr)

Epoch 1/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5024 - loss: 0.6963 - val_accuracy: 0.5000 - val_loss: 0.6938
Epoch 2/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5022 - loss: 0.6941 - val_accuracy: 0.4991 - val_loss: 0.6935
Epoch 3/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5014 - loss: 0.6936 - val_accuracy: 0.5169 - val_loss: 0.6924
Epoch 4/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4911 - loss: 0.6968 - val_accuracy: 0.5000 - val_loss: 0.6936
Epoch 5/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4946 - loss: 0.6936 - val_accuracy: 0.5009 - val_loss: 0.6934
Epoch 6/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.4982 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6934
Saved best accuracy for current run: 0.5168855786323547 at epoch 2
Run completed on:
optimizer: adam, batch_size: 16, lr: 0.005
Best accuracy for current run: 0.5168855786323547 at epoch 2
Training ended
Epoch 1/100
534/534 ━━

In [35]:
best_accuracy

{'accuracy': 0.7016885280609131,
 'epoch': 35,
 'optimizer': 'adagrad',
 'batch_size': 64,
 'lr': 0.01}

Best model is trained with Optimizer: adagrad, Epoch: 35, Batch_size: 64, Learning_rate: 0.01

In [20]:
model, history = train_model("adagrad", 40, 64, 0.01)

Epoch 1/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.5067 - loss: 0.6949 - val_accuracy: 0.5497 - val_loss: 0.6899
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5411 - loss: 0.6892 - val_accuracy: 0.5413 - val_loss: 0.6866
Epoch 3/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5690 - loss: 0.6810 - val_accuracy: 0.5779 - val_loss: 0.6716
Epoch 4/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5987 - loss: 0.6678 - val_accuracy: 0.6098 - val_loss: 0.6587
Epoch 5/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6257 - loss: 0.6545 - val_accuracy: 0.6182 - val_loss: 0.6510
Epoch 6/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6458 - loss: 0.6425 - val_accuracy: 0.6276 - val_loss: 0.6456
Epoch 7/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6570 - loss: 0.6341 - val_accuracy: 0.6266 - val_loss: 0.6420
Epoch 8/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6676 - loss: 0.6271 - val_accuracy: 0.

In [37]:
best_model = tf.keras.models.load_model("best_model.keras")
accuracy = best_model.evaluate(X_val, y_val)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6832 - loss: 0.5830 
Test accuracy: 0.692307710647583


In [38]:
best_model.get_compile_config()

{'optimizer': {'module': 'keras.optimizers',
  'class_name': 'Adagrad',
  'config': {'name': 'adagrad',
   'learning_rate': 0.009999999776482582,
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'loss_scale_factor': None,
   'gradient_accumulation_steps': None,
   'initial_accumulator_value': 0.1,
   'epsilon': 1e-07},
  'registered_name': None},
 'loss': 'binary_crossentropy',
 'loss_weights': None,
 'metrics': ['accuracy'],
 'weighted_metrics': None,
 'run_eagerly': False,
 'steps_per_execution': 1,
 'jit_compile': False}

#### Mean pooling

In [40]:
from tensorflow.keras.layers import AveragePooling1D, GlobalAveragePooling1D
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_mean.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        GlobalAveragePooling1D(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [41]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
131/134 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4601 - loss: 0.6998
Epoch 1: val_accuracy improved from -inf to 0.48311, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4605 - loss: 0.6997 - val_accuracy: 0.4831 - val_loss: 0.6947
Epoch 2/100
128/134 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4787 - loss: 0.6953
Epoch 2: val_accuracy improved from 0.48311 to 0.49062, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4793 - loss: 0.6953 - val_accuracy: 0.4906 - val_loss: 0.6933
Epoch 3/100
128/134 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4952 - loss: 0.6938
Epoch 3: val_accuracy improved from 0.49062 to 0.50750, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4956 - loss: 0.6938 - val_accuracy: 0.5075 - val_loss: 0.6922
Epoch 4/100
130/134 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5123 - loss: 0.6927
Epoch 4: val_accuracy improved fro

In [43]:
best_model = tf.keras.models.load_model("model_mean.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6029 - loss: 0.6595
Test accuracy: 0.6594746708869934


#### Max pooling

In [44]:
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPooling1D
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_max.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        GlobalMaxPooling1D(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [45]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
130/134 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5151 - loss: 0.7030
Epoch 1: val_accuracy improved from -inf to 0.53565, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5150 - loss: 0.7028 - val_accuracy: 0.5356 - val_loss: 0.6869
Epoch 2/100
127/134 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5330 - loss: 0.6903
Epoch 2: val_accuracy improved from 0.53565 to 0.55629, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5328 - loss: 0.6903 - val_accuracy: 0.5563 - val_loss: 0.6838
Epoch 3/100
124/134 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5487 - loss: 0.6866
Epoch 3: val_accuracy improved from 0.55629 to 0.57129, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5487 - loss: 0.6866 - val_accuracy: 0.5713 - val_loss: 0.6805
Epoch 4/100
128/134 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5630 - loss: 0.6823
Epoch 4: val_accuracy improved from 0

In [68]:
best_model = tf.keras.models.load_model("model_max.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7174 - loss: 0.5568
Test accuracy: 0.7514071464538574


#### Dense layer

In [55]:
from tensorflow.keras.layers import Flatten
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_dense.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        Flatten(),
        Dense(62, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [56]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.5301 - loss: 0.7015
Epoch 1: val_accuracy improved from -inf to 0.54128, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 26s 185ms/step - accuracy: 0.5303 - loss: 0.7014 - val_accuracy: 0.5413 - val_loss: 0.6979
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.6130 - loss: 0.6606
Epoch 2: val_accuracy improved from 0.54128 to 0.56848, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 145ms/step - accuracy: 0.6131 - loss: 0.6605 - val_accuracy: 0.5685 - val_loss: 0.7107
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.6440 - loss: 0.6323
Epoch 3: val_accuracy improved from 0.56848 to 0.60413, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 147ms/step - accuracy: 0.6440 - loss: 0.6322 - val_accuracy: 0.6041 - val_loss: 0.6806
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.6822 - loss: 0.6002
Epoch 4: val_accu

In [57]:
best_model = tf.keras.models.load_model("model_dense.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6231 - loss: 0.7368
Test accuracy: 0.7232645153999329


: 

# Part 3. Enhancement


#### 1. Instead of keeping the word embeddings fixed, now update the word embeddings (the same way as model parameters) during the training process.


In [ ]:
tf.random.set_seed(0)
np.random.seed(0)
random.seed(0)
model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              trainable=True),  # Embedding layer is frozen
    SimpleRNN(16, return_sequences=False),
    Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01) #Static learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping]
)

#### 2. As discussed in Question 1(c), apply your solution in mitigating the influence of OOV words and train your model again.

In [ ]:
vocab_size = len(model.index_to_key) + 1
embedding_dim = model.vector_size
word_index = {word: index+1 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [ ]:
for word, idx in word_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]
embedding_matrix[-1] = np.zeros(embedding_dim) #Add last idx as UNK word

In [ ]:
def tokenize(text, word_index):
    ls = nltk.word_tokenize(text)
    return [word_index[word] for word in ls if word in word_index else vocab_size+1]

X_train = [tokenize(text, word_index) for text in train_dataset['text']]
X_val = [tokenize(text, word_index) for text in validation_dataset['text']]
X_test = [tokenize(text, word_index) for text in test_dataset['text']]
max_length = max(len(seq) for seq in X_train)

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

In [ ]:
y_train = np.array(train_dataset['label'])
y_val = np.array(validation_dataset['label'])
y_test = np.array(test_dataset['label'])